In [8]:
import json
import glob

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit import execute
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from qiskit.quantum_info import random_clifford

import random
from qiskit.circuit.library import HGate, SdgGate
from qiskit.circuit import ClassicalRegister

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from qiskit import QuantumCircuit
from qiskit.circuit.library import U3Gate, CZGate, PhaseGate, CXGate
from mbd_utils import construct_random_clifford, cal_z_exp, calc_imbalance, cal_all_z_exp, construct_mbl_circuit, generate_disorder
from gnn import ExpValCircuitGraphModel

In [9]:
backend = FakeLima()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(FakeLima()) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [67]:
from collections import defaultdict

num_qubit = 4

def construct_random_circuit(two_q_gate_depth):
    if not two_q_gate_depth:
        d = np.random.randint(1, 6)
        rc = random_circuit(num_qubit, d, 1, measure=True)
    elif 1 <= two_q_gate_depth <= 8:
        rc = random_circuit(num_qubit, two_q_gate_depth*2, 2, measure=True)
        while rc.depth(filter_function=lambda x: x[0].num_qubits == 2) != two_q_gate_depth:
            rc = random_circuit(num_qubit, two_q_gate_depth*2, 2, measure=True)
    elif 9 <= two_q_gate_depth <= 18:
        assert two_q_gate_depth%2 == 0
        half_depth = two_q_gate_depth // 2
        rc1 = random_circuit(num_qubit, two_q_gate_depth, 2, measure=False)
        while rc1.depth(filter_function=lambda x: x[0].num_qubits == 2) != half_depth:
            rc1 = random_circuit(num_qubit, two_q_gate_depth, 2, measure=False)
        rc2 = random_circuit(num_qubit, two_q_gate_depth, 2, measure=False)
        while rc2.depth(filter_function=lambda x: x[0].num_qubits == 2) != half_depth:
            rc2 = random_circuit(num_qubit, two_q_gate_depth, 2, measure=False)
        rc = rc1.compose(rc2)
        rc.measure_all()
    else:
        raise NotImplementedError
    return rc

# depths = [2, 7, 12, 17, 22, 27, 32, 37, 42, 47]
depths = [18]

# circuit = transpile(construct_random_circuit(2), backend=backend_noisy, optimization_level=3)
circuit = construct_random_circuit(6)
circuit.draw(fold=-1)


┌─────┐┌─────────────────────────┐       ┌───┐            ┌───┐       ┌───────────────┐   ┌───┐ ┌────────┐┌────────────┐┌───┐┌───────────┐                                          ┌─┐      
q_0: ───────────■──────┤ Sdg ├┤ U3(1.2057,1.847,2.7085) ├───────┤ Z ├────────────┤ Z ├───────┤1              ├───┤ S ├─┤0       ├┤ Ry(3.4734) ├┤ I ├┤ P(3.2981) ├──────────────────────────────────■───────┤M├──────
     ┌───┐      │      └──┬──┘└─────┬─────────────┬─────┘┌──────┴───┴───────┐┌───┴───┴────┐  │  Ryy(0.17813) │   └─┬─┘ │  Iswap │└───┬───┬────┘└───┘└──┬─────┬──┘           ┌───┐                  │       └╥┘┌─┐   
q_1: ┤ I ├──────┼─────────■─────────┤ Rx(0.57535) ├──────┤ U2(1.335,5.7715) ├┤ Rz(3.1709) ├──┤0              ├─────■───┤1       ├────┤ I ├───────■─────┤ Sdg ├──────────────┤ Z ├──────────────────┼────────╫─┤M├───
     └───┘      │                   └────┬────┬───┘      └──────────────────┘└────────────┘  └─┬───────────┬─┘   ┌───┐ └─┬───┬──┘    └───┘       │     ├─────┤   ┌──────────┴───┴───────────┐┌─────┴──────┐ ║ └╥┘┌─┐
q_2: ───────────■─────────X──────────────┤ √X ├───────────────────■────────────────■───────────┤ P(4.9428) ├─────┤ H ├───┤ X ├─────────■─────────┼─────┤ Tdg ├───┤ U(3.5036,1.0148,0.67844) ├┤ Rz(4.5247) ├─╫──╫─┤M├
          ┌───────────┐   │              ├───┬┘             ┌─────┴──────┐   ┌─────┴──────┐┌───┴───────────┴───┐┌┴───┴┐  ├───┤       ┌─┴─┐       │     ├─────┤   └──────────┬────┬──────────┘└────┬─┬─────┘ ║  ║ └╥┘
q_3: ─────┤ P(3.3446) ├───X──────────────┤ I ├──────────────┤ Rz(4.7053) ├───┤ Rx(4.1697) ├┤ U2(5.5983,1.7274) ├┤ Tdg ├──┤ S ├───────┤ H ├───────■─────┤ Tdg ├──────────────┤ √X ├────────────────┤M├───────╫──╫──╫─
          └───────────┘                  └───┘              └────────────┘   └────────────┘└───────────────────┘└─────┘  └───┘       └───┘             └─────┘              └────┘                └╥┘       ║  ║  ║ 
c: 4/══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩════════╩══╩══╩═
                                                                                                                                                                                                   3        0  1  2

In [68]:
circuit.depth(lambda x: x[0].num_qubits == 2)

6

In [63]:
from qiskit import QuantumRegister, ClassicalRegister
qr = QuantumRegister(2)
qc = QuantumCircuit(qr)
qc.cz(0, 1)
qc.depth(lambda x: x[0].num_qubits == 2)

1

In [7]:
generate_data(construct_random_circuit, depths, './data/haoran_mbl/rc_train', 500)

  0%|          | 0/500 [00:00<?, ?it/s]

In [24]:
f = open('./data/haoran_mbl/rc_train/random_circuits_depth4.json')
out = json.load(f)

In [25]:
out[1]['circuit_depth']

36